In [7]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy.signal as sc

In [ ]:
import time
import threading

class PeriodicSleeper(threading.Thread):
    def __init__(self, task_function, period):
        super().__init__()
        self.task_function = task_function
        self.period = period
        self.i = 0
        self.t0 = time.time()
        self.start()

    def sleep(self):
        self.i += 1
        delta = self.t0 + self.period * self.i - time.time()
        if delta > 0:
            time.sleep(delta)
    
    def run(self):
        while True:
            self.task_function()
            self.sleep()

In [ ]:
def task():
    t = time.time()
    if abs(t - round(t)) <= 0.0005:
        print(t, 'Mean Frequency:', sleeper.i / (t - sleeper.t0))

sleeper = PeriodicSleeper(task, 0.001)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt


s = np.random.poisson(1, 10000)
import matplotlib.pyplot as plt
count, bins, ignored = plt.hist(s, 101, density=True)
plt.show()
#print(10*count)
print(len(count))

In [ ]:
import pandas as pd
file = '/Users/francescostraniero/Documents/Pembroke_4th_year/MPhys_Proj/myfile.csv'
df = pd.read_csv(file)
df.iloc[:,0]=0
lenght = df.shape[1]
for i in df.columns:

    y = df[str(i)]
    x = np.linspace(0,len(y),len(y))
    plt.plot(x,y)

plt.xlabel('time (10*ms)')
plt.ylabel('Transformed signal intensity (a.u.)')
'''
for i in range(50,60):
    y = df[str(i)]
    x = np.linspace(0,len(y),len(y))
    plt.plot(x,y)
'''

In [ ]:
y = df.iloc[:,3]
x = np.linspace(0,len(y),len(y))
plt.plot(x,y)
plt.xlabel('time (10*ms)')
plt.ylabel('Transformed signal intensity (a.u.)')
plt.title('Transformed signal with lower pulse intensity')

In [ ]:

timings = []
for i in df.columns:
    l = float(i)
    timings.append(float(i))
print(timings)

diffs = []

for i in range(len(timings)-1):
    diffs.append(timings[i+1]-timings[i])

print(diffs)

In [ ]:
r = np.random.exponential(1, size=50)

print(np.mean(r))
count, bins, errors = plt.hist(r,bins = 100)

In [ ]:
from adc_filter import adc_filter_func

In [ ]:
arr = np.zeros([100,100])

for i in range(100):
    arr[i,:] = i

print(arr)
arr[:,-1]

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
x= np.zeros(1000)
photo_electrons = np.zeros(100)
s = np.random.exponential(1, 1000)
count, bins = np.histogram(s, 100, density=True)
count = 900 * count
photo_electrons = np.add(photo_electrons, count)
y_vals = np.linspace(599, 699, 100, dtype=int)

for idx in y_vals:
    x[idx] = count[idx-600] if idx < (600+len(count)) else 0

x_axis = np.linspace(0,1000,1000)
plt.plot(x_axis,x)
plt.xlabel('time(10*ms)')
plt.ylabel('Signal Intensity (a.u.)')

plt.title('Example of photoelectrons current registered at anode, after one S1 event.')

In [ ]:
zeros = np.zeros(20)
sample= np.append(zeros,x)
sample = np.append(sample,zeros)
sample = np.append(sample,x)

x_axis = np.linspace(0,len(sample),len(sample))

plt.plot(x_axis,sample)

In [ ]:
y = adc_filter_func(sample,[0.012,0.02])
plt.plot(x_axis,y, linewidth = 0.4)


In [ ]:
from S1_event_generator import S1_event


y, numb, pos= S1_event(5)

plt.plot(np.arange(0,len(y),1),y)

In [ ]:
import scipy.signal as sc

noise_std = np.std(y[0:1000])
noise_mean = np.mean(y[0:1000])
pod_threshold = noise_mean+(5.9*noise_std)
peaks, props = sc.find_peaks(y, height=pod_threshold,prominence=7,wlen=10)
print(noise_std)
print(peaks)
print(numb)
print(pos)

In [ ]:
s = [peaks[0]]

for i in range(len(peaks)-1):
    if peaks[i+1]-peaks[i]<500:
        start =peaks[i]
        stop=peaks[i+1]
            
        if np.min(y[start:stop])<1.5*pod_threshold:
            s.append(peaks[i])

print(s)


In [ ]:
peaks_cwt = sc.find_peaks_cwt(y,widths=60)

print(peaks_cwt)
print(numb)
print(pos)

In [ ]:
y_max = np.max(y)

refined_peaks_cwt =[]
for i in range(len(peaks_cwt)):
    if y[peaks_cwt[i]] > 0.3*y_max:
        refined_peaks_cwt.append(peaks_cwt[i])
        
print(refined_peaks_cwt)



In [8]:
def peak_identifier(array):
    peaks_cwt = sc.find_peaks_cwt(y,widths=60)

    y_max = np.max(y)
    refined_peaks_cwt =[]

    for i in range(len(peaks_cwt)):
        if y[peaks_cwt[i]] > 0.3*y_max:
            refined_peaks_cwt.append(peaks_cwt[i])
    return(refined_peaks_cwt)


In [9]:
from S1_event_generator import S1_event


number_of_events = 15
results = np.empty((2,number_of_events))


for i in range(100):
    y, numb, pos= S1_event(number_of_events)

    measured_peaks = peak_identifier(y)

    if len(pos) == number_of_events:
        results = np.vstack((results, pos))
    else:
        extras_zeros = number_of_events-len(pos)
        pos = np.append(pos,np.zeros(extras_zeros))
        results = np.vstack((results, pos))

    
    if len(measured_peaks)==number_of_events:
        results = np.vstack((results, measured_peaks))
    else:
        extra_zeros = number_of_events-len(measured_peaks)
        measured_peaks = np.append(measured_peaks,np.zeros(extra_zeros))
        results = np.vstack((results, measured_peaks))
    



In [ ]:
print(pos)

In [10]:
results1 = np.delete(results,(0,1),0)
df = pd.DataFrame(results1)

file_path = '/Users/francescostraniero/Documents/Pembroke_4th_year/MPhys_Proj/100events_15peaks2.csv'
df.to_csv(file_path,mode='w',index = False)
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,2000.0,2031.0,2182.0,2256.0,2311.0,2433.0,2664.0,3153.0,3301.0,4423.0,5013.0,6234.0,6483.0,6939.0,11999.0
1,2077.0,2337.0,3208.0,3366.0,4490.0,5080.0,6299.0,6552.0,7005.0,12065.0,0.0,0.0,0.0,0.0,0.0
2,2000.0,2023.0,2030.0,2162.0,2164.0,2644.0,2776.0,2899.0,3089.0,3114.0,3259.0,3482.0,3910.0,4204.0,11999.0
3,2077.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2000.0,2214.0,2274.0,2994.0,3166.0,3280.0,3425.0,3737.0,3764.0,3782.0,3882.0,4430.0,6077.0,6484.0,11999.0
